#    Tensor Trade with LSTM series prediction

In [ ]:
##### Collection of thoughts and notes about my experience trying to beat the market. 

In [ ]:
Resources:
GITHUB Author @NotAdamKing
https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline 
https://discordapp.com/channels/592446624882491402/593538654857723909
https://towardsdatascience.com/trade-smarter-w-reinforcement-learning-a5e91163f315 

GITHUB Author @hootnuot
https://github.com/hootnot/oandapyV20-examples 
http://developer.oanda.com/rest-live-v20/instrument-ep/

GITHUB Author @philipperemy
https://github.com/philipperemy/FX-1-Minute-Data/blob/master/download_all_fx_data.py 

https://en.wikipedia.org/wiki/Sortino_ratio


In [ ]:
Get sources for Forex: 
    We can add more exchanges to the tickers list which is used throughout the modules.
Note: Most of the tickers are in the {CUR_CUR} format. This snippet needs the "EURUSD" so we need to replace it. Data is stored in a local SQL DB with the bid and ask price. Currently, we use the ask and resample this in the OHLC format. 

In [ ]:
#https://github.com/philipperemy/FX-1-Minute-Data/blob/master/download_all_fx_data.py GITHUB Author @philipperemy
from api import download_fx_m1_data, download_fx_m1_data_year
 tickers = ["EUR_USD"]
    for ticker in tickers:
        exchange = ticker
        ticker = ticker.replace("_", "")
        year = int(2018)
        print(ticker)
        output_folder = os.path.join('output', ticker)
        mkdir_p(output_folder)
        try:
            while True:
                could_download_full_year = False
                try:
                    output_filename = download_fx_m1_data_year(year, ticker)
                    shutil.move(output_filename, os.path.join(output_folder, output_filename))
                    could_download_full_year = True
                except:
                    pass  # lets download it month by month.
                month = 1
                while not could_download_full_year and month <= 12:
                    output_filename = download_fx_m1_data(str(year), str(month), ticker)
                    shutil.move(output_filename, os.path.join(output_folder, output_filename))
                    month += 1
                year += 1

In [ ]:
Stock	Ask	Date
EUR_USD	1.10372	10/11/2019 16:39
EUR_USD	1.10372	10/11/2019 16:40
EUR_USD	1.10372	10/11/2019 16:41
EUR_USD	1.10362	10/11/2019 16:42
EUR_USD	1.10358	10/11/2019 16:43
EUR_USD	1.10359	10/11/2019 16:44
EUR_USD	1.10358	10/11/2019 16:45
EUR_USD	1.10354	10/11/2019 16:47
EUR_USD	1.10353	10/11/2019 16:48
EUR_USD	1.10353	10/11/2019 16:49
EUR_USD	1.10356	10/11/2019 16:50
EUR_USD	1.10355	10/11/2019 16:51
EUR_USD	1.10358	10/11/2019 16:52
EUR_USD	1.10363	10/11/2019 16:53
EUR_USD	1.10357	10/11/2019 16:54
EUR_USD	1.10348	10/11/2019 16:55
EUR_USD	1.10343	10/11/2019 16:56
EUR_USD	1.10353	10/11/2019 16:57
EUR_USD	1.10363	10/11/2019 16:58

In [ ]:
Random samples are chosen for training and reverse sorted. This is to ensure that we are training on real-world shuffled data.

In [ ]:
df = pd.read_sql("select * from (select date as Date, ask as Open, ask as Low, ask as High, ask as Close from forex_daily_price where exchange = '{}' and date between '{}' and '{}'  ORDER BY date DESC LIMIT {}) tmp order by tmp.date ASC".format(ticker, s_date, e_date, limit), conn)
resampled = grouped_data[target].resample(period).ohlc()
resampled['Date'] = pd.to_datetime(resampled['Date'], format="%Y-%m-%d %H:%M:%S").astype('datetime64[ns]').view('int64')
#target is the exchange we wish to train on.
#period is the period we wish to train (ie 5T, 10T, 15T etc.)


In [ ]:
 "Load a random param for training. Currently, I don't find that one data set does better than the next. For each iteration, I load a random param and save the highest score. There are 15 total iterations with randomly chosen params."

In [ ]:
After each training episode is complete I run a test on another smaller sample set; save and assign an overall score. The lowest scores are deleted leaving only the top 5
The accuracy is measured using with a test sample and predicting the next Nth in series. We look ahead and measure the percent difference from the market.

Test results:
    Mean error: 0.000061
    Accuracy: 0.888671

##For use with live data
#http://developer.oanda.com/rest-live-v20/instrument-ep/
#https://github.com/hootnot/oandapyV20-examples
#gather the last 1000 points ie: (period * 1000)
from oandapyV20 import API
for i in self.clargs.instruments:
                r = instruments.InstrumentsCandles(instrument=i, params=params)
                rv = self.api.request(r)
                kw = {}
                if self.clargs.nice:
                    kw = {"indent": self.clargs.nice}
                print("{}".format(json.dumps(rv, **kw)))

In [ ]:
once training accuracy is above the threshold, it will automatically begin backtesting the SQL DB. Updating the results. The new table will look like: 
    Date|Open|High|Low|Close|5T_Open|5T_High|5T_Low|5T_Close|5T_Confidence|10T_Open|10T_High|10T_Low|10T_Close|10T_Confidence|15T_Open|15T_High|15T_Low|15T_Close|15T_Confidence  
    I have noticed when the candles flatten out the accuracy of the predictions will drop tremendously. This works best when the data is volatile.
    I can test the data by the following formula (actual * confidence) - prediction This should be very close to 0.
    I may add this formula to an SQL query at a later date. For now I load a sample set into excel and check it with the described formula.
    example: actual value was 1.00315 and predicted value was 1.002899~ or 0.0250% error  #sample for p-5T with Open
    =((1.002899 - 1.00315) / 1.00315) / 1.00315

I may add in more features if I see an increase with the newly added features. 
My control ran for ep: 1000 : st: 1000 with a reward of -0.0068
#Current status of new features is null. 
    

In [ ]:
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features.stationarity import FractionalDifference
from tensortrade.features import FeaturePipeline
from tensortrade.rewards.risk_adjusted_return_strategy import RiskAdjustedReturnStrategy
from tensortrade.actions import DiscreteActionStrategy
from tensortrade.environments import TradingEnvironment
from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import A2C
from tensortrade.strategies import StableBaselinesTradingStrategy
import pandas as pd
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features import FeaturePipeline
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features.stationarity import FractionalDifference
from tensortrade.features.indicators import SimpleMovingAverage
import mysql.connector

reward_strategy = RiskAdjustedReturnStrategy()
action_strategy = DiscreteActionStrategy(n_actions=20,
                                        instrument_symbol='ETH/BTC')

#Thought: Adjust n_actions by the confidence score for that metric. Set a maximum for lower scores and a minimum for higher scores

model = A2C
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5 }
performance = strategy.run(steps=10000, episodes=10000)
#running multiple steps, multiple episodes as I have read other research papers claiming better results.
strategy.save_agent(path="a2c_btc_1h")
normalize_price = MinMaxNormalizer(["Date","Open","High","Low","Close","5T_Open","5T_High","5T_Low","5T_Close","5T_Confidence","10T_Open","10T_High","10T_Low","10T_Close","10T_Confidence","15T_Open","15T_High","15T_Low","15T_Close","15T_Confidence" ])
#I dont think I need to normalize all my columns; I will try a sample to see how things go.
strategy.save_agent(path="data/a2c_{}_{}_{}_{}".format(ticker, period, steps, episodes))


In [ ]:
@isaksson delux
trading fees depend on the platform you want to trade on, so they vary. you might include it in your reward function. same goes for funding rate if u want to simulate margin
 
@Chuba
They also vary by trade type
Limits vs market order.

Need to look into this closer as this can break the best trade stategy. 

In [ ]:
class RiskAdjustedReturnStrategy(RewardStrategy):
    returns = self._exchange.performance['net_worth'].diff()
    risk_adjusted_return = self._return_algorithm(returns)

In [ ]:
When I trade manually I wait until the price crosses over the support or resistance line. This indicates the price is about to break out an go up or down. This may make a good parameter to add to our model

In [ ]:
#I'm not the creator of this snippet but cannot site the original owner.
def get_upper_lower():
	df = pd.read_sql("select * from (select date, ask from forex_daily_price where exchange = '{}' ORDER BY date DESC LIMIT {}) tmp order by tmp.date ASC".format(ticker, limit), conn)
	# group by day and drop NA values (usually weekends)
	ticks_data = grouped_data['Sell'].resample('1min').ohlc()
	# use 'ask'
	sell_data = grouped_data.as_matrix(columns=['Sell'])
	# calculate bandwidth (expirement with quantile and samples)
	bandwidth = estimate_bandwidth(sell_data, quantile=0.1, n_samples=100)
	ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
	
	# fit the data
	ms.fit(sell_data)
	
	ml_results = []
	for k in range(len(np.unique(ms.labels_))):
		my_members = ms.labels_ == k
		values = sell_data[my_members, 0]
		
		# find the edges
		lower = ml_results.append(min(values))
		upper = ml_results.append(max(values))
    return lower, upper